In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import pandas as pd
from functions import main_cleaning

pd.set_option('display.max_columns', None) 

df_raw = pd.read_csv("dielectron.csv")

In [33]:
df_cleaned = main_cleaning(df_raw)

e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_same_charge"] = df["Q1"] == df["Q2"]
e:\Iván\Documentos\Clase\Informatica\Beca IronHack\Temario\PROYECTOS\Proyecto_final_1710\functions.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_outlier"] = False


In [34]:
df_cleaned = df_cleaned.dropna()         # Solo lo hago en el df_cleaned de ML

Results sin normnalizar -> (17.717622715597013, 12.581512831873468, 0.507506217561434)

In [35]:
features = df_cleaned[["Run", "E1", "E2", "pt1", "pt2", "is_same_charge", "is_outlier"]]
target = df_cleaned[["M"]]

In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.20, random_state=42)

In [37]:
from sklearn.preprocessing import MinMaxScaler

normalizer = MinMaxScaler()

X_train_norm= normalizer.fit_transform(X_train)        

X_test_norm = normalizer.transform(X_test)

In [38]:
X_train_norm = pd.DataFrame(X_train_norm, columns = X_train.columns)   
X_test_norm = pd.DataFrame(X_test_norm, columns = X_test.columns)

In [39]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.metrics import root_mean_squared_error, mean_absolute_error, r2_score

# Define models
models = {
  'Linear Regression': LinearRegression(),
  'Decision Tree': DecisionTreeRegressor(random_state=42),
  'Random Forest': RandomForestRegressor(random_state=42, n_estimators=100),
  'SVR': SVR(),
}   

# Train and evaluate models
results = {}

for name, model in models.items():
  # Train the model
    model.fit(X_train_norm, y_train)
  
  # Make predictions
    y_pred_scaled = model.predict(X_test_norm)

    mae_standardized = mean_absolute_error(y_test, y_pred_scaled)
    mse_standardized = root_mean_squared_error(y_test, y_pred_scaled)
    r2_standardized = r2_score(y_test, y_pred_scaled)
  
  # Store results
    results[name] = {
        'mae' : mae_standardized,
        'mse' : mse_standardized,
        'R2' : r2_standardized
  }

# Print results
for name, result in results.items():
  print(f"\n{name}:")
  print(f"Mean Absolute Error: {result['mae']}")
  print(f"Root Mean Squared Error: {result['mse']}")
  print(f"R2: {result['R2']}")

# Compare R2_value
r2_values = {name: result['R2'] for name, result in results.items()}
best_model = max(r2_values, key=r2_values.get)

print("\nModel Accuracy Comparison:")
for name, accuracy in r2_values.items():
  print(f"{name}: {accuracy:.4f}")

print(f"\nBest performing model: {best_model} with accuracy {r2_values[best_model]:.4f}")

c:\Anaconda\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Anaconda\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)



Linear Regression:
Mean Absolute Error: 14.250262626471216
Root Mean Squared Error: 19.567297812140065
R2: 0.3993083941460204

Decision Tree:
Mean Absolute Error: 14.432126316131937
Root Mean Squared Error: 23.7915225621365
R2: 0.11195632536880129

Random Forest:
Mean Absolute Error: 12.093365967736123
Root Mean Squared Error: 17.093239655715877
R2: 0.5416063249821053

SVR:
Mean Absolute Error: 13.105359066991904
Root Mean Squared Error: 19.63691366864826
R2: 0.39502655106727325

Model Accuracy Comparison:
Linear Regression: 0.3993
Decision Tree: 0.1120
Random Forest: 0.5416
SVR: 0.3950

Best performing model: Random Forest with accuracy 0.5416
